# Visualize and Analyze the PV capacities in Minden

requires `renewable_power_plants_DE.csv` from [Open Power System Data](https://data.open-power-system-data.org/renewable_power_plants/2020-08-25/renewable_power_plants_DE.csv) <br>
and `pv_plants_minden.csv` from [Markstammdatenregister](https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/ErweiterteOeffentlicheEinheitenuebersicht) <br>
in the `/data/` folder 

## Imports

In [ ]:
import atlite
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Load data

In [ ]:
# load csv file into pandas DataFrame
df_renewables = pd.read_csv("../data/renewable_power_plants_DE.csv", sep=",", decimal=".", header=0)
df_renewables

In [ ]:
df_renewables.columns

## Visualize PV capacities

In [ ]:
# filter for solar renewables
df_pv = df_renewables[df_renewables["energy_source_level_2"] == "Solar"]

In [ ]:
map_germany = gpd.read_file("../data/DEU_adm1.shp")
map_germany.index = (["Baden-Würtemberg", "Bayern", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hessen", "Mecklenburg-Vorpommern", "Niedersachsen", "Nordrhein-Westfalen", "Rheinland-Pfalz", "Saarland", "Sachsen-Anhalt", "Sachsen", "Schleswig-Holstein", "Thüringen"])
map_germany.plot()

In [ ]:
df_pv_sum = df_pv.groupby("federal_state")["electrical_capacity"].sum()
df_pv_sum = df_pv_sum[df_pv_sum.index != "Ausland"]
merged = map_germany.merge(df_pv_sum, left_index=True, right_index=True)
merged

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 6))
merged.plot(column="electrical_capacity", cmap='YlOrRd', ax=ax)
ax.axis('off')
ax.set_title('PV Kapazität pro Bundesland in MW - 2020')
sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=merged["electrical_capacity"].min(), vmax=merged["electrical_capacity"].max()))
sm._A = []
cbar = fig.colorbar(sm, ax=ax)

In [ ]:
# filter for all pv plants in minden
df_pv_minden = df_pv[df_pv["municipality"] == "Minden"]
df_pv_minden

In [ ]:
# select only the relevant columns
df_pv_minden = df_pv_minden[["electrical_capacity", "technology", "data_source", "lon", "lat", "commissioning_date", "voltage_level"]]

In [ ]:
df_pv_minden

In [ ]:
(df_pv_minden["electrical_capacity"] * 1000).describe()

In [ ]:
df_pv_minden["electrical_capacity"].sum()

## Calculate PV feed-ins for Minden

In [ ]:
df_pv_plants_minden = pd.read_csv("../data/pv_plants_minden.csv", header=0, sep=";", decimal=",")

In [ ]:
df_pv_plants_minden.columns

In [ ]:
df_pv_plants_minden = df_pv_plants_minden[["Nettonennleistung der Einheit", "Anzahl der Solar-Module", "Hauptausrichtung der Solar-Module"]]
df_pv_plants_minden

In [ ]:
azimuth_mapping = {
        "Nord" : 0.0,
        "Nord-Ost" : 45.0,
        "Ost" : 90.0,
        "Süd-Ost" : 135.0,
        "Süd" : 180.0,
        "Süd-West": 225.0,
        "West": 270.0,
        "Nord-West": 315.0
    }

In [ ]:
# add coordinates from Minden
df_pv_plants_minden["x"] = 8.9167
df_pv_plants_minden["y"] = 52.2833
df_pv_plants_minden["azimuth"] = df_pv_plants_minden["Hauptausrichtung der Solar-Module"].map(azimuth_mapping)
df_pv_plants_minden = df_pv_plants_minden.dropna()
df_pv_plants_minden["capacity"] = df_pv_plants_minden["Nettonennleistung der Einheit"]
df_pv_plants_minden = df_pv_plants_minden.drop(columns=["Nettonennleistung der Einheit", "Anzahl der Solar-Module", "Hauptausrichtung der Solar-Module"])
df_pv_plants_minden

In [ ]:
# load weather data cutout
weather_minden = atlite.Cutout("../data/minden-2022.nc")

In [ ]:
azimuth_mapping = {
    "Nord" : 0.0,
    "Nord-Ost" : 45.0,
    "Ost" : 90.0,
    "Süd-Ost" : 135.0,
    "Süd" : 180.0,
    "Süd-West": 225.0,
    "West": 270.0,
    "Nord-West": 315.0
}
azimuth_mapping_inverted = {v: k for k, v in azimuth_mapping.items()}

azimuths = df_pv_plants_minden["azimuth"].unique()
feed_ins = []

for az in azimuths:
    capacities = df_pv_plants_minden[df_pv_plants_minden["azimuth"] == az]
    solar_layout = weather_minden.layout_from_capacity_list(capacities, col="capacity")
    pv_feed_in = weather_minden.pv(
        panel=atlite.solarpanels.CSi, orientation={"slope": 30.0, "azimuth": az}, layout=solar_layout
    )
    pv_feed_in_series = pv_feed_in.squeeze().to_series()
    pv_feed_in_series.name = f"{azimuth_mapping_inverted[az]}_kW"
    feed_ins.append(pv_feed_in_series)

# concat time series per azimuth into one dataframe
results = pd.concat(feed_ins, axis=1)

# sum all azimuth feed-ins per timestep
results["SUM"] = results.sum(axis=1)

results = results.tz_localize('UTC')
results = results.tz_convert('Europe/Berlin')

results

In [ ]:
fig = go.Figure()
# visualize feed-ins per azimuth as bars
for az in results.columns[:-1]:
    fig.add_trace(go.Bar(x=results.index, y=results[az], name=az.replace("_kW", "")))

# show the sum as a line
fig.add_trace(go.Scatter(x=results.index, y=results["SUM"], name="Einspeisung Gesamt"))

fig.update_layout(barmode="stack", title_text=f"PV Einspeisungen Minden <br> 2022", xaxis_title="Datum", yaxis_title="Einspeisung [kW]", legend_title="Ausrichtungen")
fig.show()